In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import time
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import pandas as pd
from scipy.interpolate import griddata
from MSNN_GPU import MultistageNeuralNetwork
from utils import NeuralNet, create_ds, poisson

2025-04-09 13:38:10.092434: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def load_model(file_path, x_train, y_train):
    with open(file_path, 'rb') as f:
        model_data = pickle.load(f)
        
    model = MultistageNeuralNetwork(x_train, len(model_data['layers']) - 2, model_data['layers'][1])
    model.dim = model_data['dim']
    model.N = model_data['N']
    model.layers = model_data['layers']
    model.lt = model_data['lt']
    model.ut = model_data['ut']

    for stage_data in model_data['stages']:
        nn = NeuralNet(x_train, y_train, layers=model.layers, kappa=stage_data['kappa'],
            lt=stage_data['lt'], ut=stage_data['ut'], acts=0
        )
        nn.weights = [tf.Variable(w, dtype=tf.float64) for w in stage_data['weights']]
        nn.biases = [tf.Variable(b, dtype=tf.float64) for b in stage_data['biases']]
        model.stages.append(nn)

    return model

In [ ]:
def create_data(N_train, dim):
    x_train = create_ds(dim, -1.02, 1.02, N_train)
    y_train = tf.reshape(poisson(x_train), [len(x_train), 1])
    return x_train, y_train

x_one, y_one = create_data(20, 1)
x_five, y_five = create_data(15, 5)
x_eight, y_eight = create_data(6, 8)
x_ten, y_ten = create_data(2, 10)